# Overall summary

This notebook provides an **overall summary** of the sharing of models against our best practices for enabling other to use and execute the simulation model.  In summary this is defined as:

## Data used in analysis

The dataset is a subset of the main review - limited to models shared.  The type of model shared is coded as **Visual Interactive Modelling (VIM)** based (e.g Anylogic, Simul8, Arena) versus **CODE** (e.g. Matlab, Python, SimPy, Java, R Simmer).

> The data can be found here: https://raw.githubusercontent.com/TomMonks/des_sharing_lit_review/main/data/bp_audit.zip

The following fields are analysed in this notebook.

* `model_format` - VIM or CODE
* `model_has_doi`  - do the model artefacts have their own minted DOI? (0/1)
* `orcid` - do the researchers provide an ORCID with the model? (0/1)
* `license` - does the model have an explicit license defining how it can be used? (str)
* `readme` - is there an obvious file(s) where a user would look first? (0/1)             
* `steps_run` - are there steps to run a model? (0/1)
* `formal_dep_mgt` - has the model been shared with formal software dependency management? (0/1)
* `informal_dep_mgt` - have any informal methods of dependency management been shared?  E.g. a list of software requirements. (0/1)
* `evidence_testing` - do the model and artefacts in the repository contain any evidence that they have been tested? (0/1)
* `downloadable` - can the model and artefacts be downloaded and executed locally? (0/1)
* `interactive_online` - can the model and its artefacts be executed online without local installation? (0/1)

## 1. Imports

### 1.1. Standard

In [1]:
import pandas as pd
import numpy as np

### 1.2 Preprocessing

In [2]:
from preprocessing import load_clean_bpa, drop_columns

## 2. Constants

In [3]:
FILE_NAME = 'https://raw.githubusercontent.com/TomMonks/' \
    + 'des_sharing_lit_review/main/data/bp_audit.zip'

## 3. Analysis functions

A number of simple functions to conduct the analysis and format output.

In [4]:
def balance_of_model_format(df):
    '''
    REturns the counts of VIM versus code
    
    Params:
    -------
    df: pd.DataFrame
        Subset of the best practice dataset to analyse
        
    Returns:
    (labels: List, counts: List)
    '''
    unique_elements, counts_elements = np.unique(df['model_format'], 
                                                 return_counts=True)
    return unique_elements, counts_elements

In [5]:
def category_frequencies_by_model_format(df, cols):
    '''
    Calculate the frequencies of 0/1s for VIM versus code.
    Return concatenated in a pandas dataframe
    
    Params:
    ------
    df: pd.DataFrame
        DAtaframe containing subset of best practice audit to summarise.
    
    Returns:
    -------
    pd.DataFrame
    '''

    # key to select fields where category is 1.
    key = [('CODE', 1), ('VIM', 1)]

    df = pd.DataFrame()

    # operation needs to be done separetly on each criteria then combined.
    for col in list(clean[cols]):
        # group by VIM and code and get frequencies of 1/0
        results = clean.groupby('model_format')[col].value_counts(dropna=False)
        # concat to single dataframe
        df = pd.concat([df, results.loc[key]], axis=1)
        
    # drop multi-index, transpose and relabel
    df = df.reset_index()
    df = df.T
    df = df.drop(['level_0', 'level_1'])
    df.columns = ['CODE', 'VIM']
    
    # add percentages
    # get total number of code and vim based models.
    _, (n_code, n_vim) = balance_of_model_format(clean)
    per_cols = ['CODE_%', 'VIM_%']
    df[per_cols[0]] = (df['CODE'] / n_code * 100).map('{:,.1f}'.format)
    df[per_cols[1]] = (df['VIM'] / n_vim * 100).map('{:,.1f}'.format)
    
    return df

In [6]:
def model_has_license(license):
    '''
    Recode the license column from multiple categories down to binary.
    None = 0 else 1
    
    Params:
    ------
    license: pd.Series
        The series containing the license info
        
    Returns:
    -------
    int
    '''
    if license == "None":
        return 0
    else:
        return 1

In [7]:
def format_bpa_results(summary):
    '''
    Convert 4 column able of n and % into two 
    columns where each column is n (%)
    
    Params:
    -------
    summary: pd.DataFrame
        The unformatted table.  Assumes 4 cols and index.
        
    Returns:
    -------
    pd.DataFrame
    '''
    
    row_headings = ['Model has DOI',
                    'ORCID',
                    'Licensed',
                    'Readme',
                    'Steps to run',
                    'Formal Dep Mgt',
                    'Informal Dep Mgt',
                    'Evidence of testing',
                    'Model downloadable',
                    'Model interactive online']
    
    summary['CODE (\%)'] = summary['CODE'].map('{:,.0f}'.format) \
        + ' (' + summary['CODE_%'] + ')'
    
    summary['VIM (\%)'] = summary['VIM'].map('{:,.0f}'.format) \
        + ' (' + summary['VIM_%'] + ')'
    
    
    summary = summary.drop(['CODE', 'VIM', 'CODE_%', 'VIM_%'], axis=1)
    summary['criteria'] = row_headings
    summary = summary.set_index('criteria')
    return summary

## 4. Load and inspect dataset

The clean data set has 27 fields included.  These are listed below.  

In [8]:
clean = load_clean_bpa(FILE_NAME)
clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   model_format                  43 non-null     category
 1   key                           43 non-null     object  
 2   item_type                     43 non-null     category
 3   pub_yr                        42 non-null     float64 
 4   author                        43 non-null     object  
 5   doi                           42 non-null     object  
 6   reporting_guidelines_mention  43 non-null     category
 7   covid                         43 non-null     category
 8   sim_software                  43 non-null     object  
 9   foss_sim                      43 non-null     category
 10  model_archive                 4 non-null      object  
 11  model_repo                    19 non-null     object  
 12  model_journal_supp            9 non-null      object

## 5. Results

### 5.1 Summary table.

The table illustrates the number of 

In [9]:
cols = ['model_has_doi', 'orcid', 'license_y', 'readme', 'steps_run', 
        'formal_dep_mgt', 'informal_dep_mgt', 
        'evidence_testing', 'downloadable', 'interactive_online']

clean['license_y'] = clean['license'].apply(model_has_license)
unformatted = category_frequencies_by_model_format(clean, cols)
unformatted

,CODE,VIM,CODE_%,VIM_%
model_has_doi,4,3,14.3,20.0
orcid,3,3,10.7,20.0
license_y,14,6,50.0,40.0
readme,20,7,71.4,46.7
steps_run,12,3,42.9,20.0
formal_dep_mgt,7,0,25.0,0.0
informal_dep_mgt,6,8,21.4,53.3
evidence_testing,3,0,10.7,0.0
downloadable,28,10,100.0,66.7
interactive_online,4,6,14.3,40.0


### 5.2 Formatted Results for paper + $\LaTeX$

In [10]:
table = format_bpa_results(unformatted)
table

,CODE (\%),VIM (\%)
criteria,,
Model has DOI,4 (14.3),3 (20.0)
ORCID,3 (10.7),3 (20.0)
Licensed,14 (50.0),6 (40.0)
Readme,20 (71.4),7 (46.7)
Steps to run,12 (42.9),3 (20.0)
Formal Dep Mgt,7 (25.0),0 (0.0)
Informal Dep Mgt,6 (21.4),8 (53.3)
Evidence of testing,3 (10.7),0 (0.0)
Model downloadable,28 (100.0),10 (66.7)


In [11]:
# output as latex
print(table.style.to_latex(hrules=True, 
                          label="Table:bpa_results", 
                          caption="Best practice audit results"))

\begin{table}
\caption{Best practice audit results}
\label{Table:bpa_results}
\begin{tabular}{lll}
\toprule
 & CODE (\%) & VIM (\%) \\
criteria &  &  \\
\midrule
Model has DOI & 4 (14.3) & 3 (20.0) \\
ORCID & 3 (10.7) & 3 (20.0) \\
Licensed & 14 (50.0) & 6 (40.0) \\
Readme & 20 (71.4) & 7 (46.7) \\
Steps to run & 12 (42.9) & 3 (20.0) \\
Formal Dep Mgt & 7 (25.0) & 0 (0.0) \\
Informal Dep Mgt & 6 (21.4) & 8 (53.3) \\
Evidence of testing & 3 (10.7) & 0 (0.0) \\
Model downloadable & 28 (100.0) & 10 (66.7) \\
Model interactive online & 4 (14.3) & 6 (40.0) \\
\bottomrule
\end{tabular}
\end{table}

